In [2]:
import pandas as pd
import numpy as np
import re
import pickle

In [3]:
df = pd.read_csv('data/data.csv', sep =';')

In [4]:
def winner (red,winner):
    if red == winner:
        return 1
    else:
        return 0

In [5]:
df['winner'] = df.apply(lambda x: winner(x['R_fighter'],x['Winner']),axis=1)

In [7]:
#subset to decisions only
df = df[df['win_by'].str.contains("Decision")]
df = df[df['Fight_type'].str.contains("Title")]
df.shape

(125, 42)

In [9]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'winner'],
      dtype='object')

In [10]:
#use the below to convert any string percents to actual percents
# df['col'] = df['col'].str.rstrip('%').astype('float') / 100.0
df['R_SIG_STR_pct'] = df['R_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['B_SIG_STR_pct'] = df['B_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['R_TD_pct'] = df['R_TD_pct'].str.rstrip('%').astype('float') / 100.0
df['B_TD_pct'] = df['B_TD_pct'].str.rstrip('%').astype('float') / 100.0


In [9]:
new1 = df["R_HEAD"].str.split("of", n = 1, expand = True) 
new2 = df["B_HEAD"].str.split("of", n = 1, expand = True) 
new3 = df["R_BODY"].str.split("of", n = 1, expand = True) 
new4 = df["B_BODY"].str.split("of", n = 1, expand = True) 
new5 = df["R_LEG"].str.split("of", n = 1, expand = True) 
new6 = df["B_LEG"].str.split("of", n = 1, expand = True) 

df['head_diff'] = new1[0].astype(float)/new1[1].astype(float) - new2[0].astype(float)/new2[1].astype(float)
df['body_diff'] = new3[0].astype(float)/new3[1].astype(float) - new4[0].astype(float)/new4[1].astype(float)
df['leg_dif'] = new5[0].astype(float)/new5[1].astype(float) - new6[0].astype(float)/new6[1].astype(float)
df['pass_diff'] = df['R_PASS'] - df['B_PASS']

In [11]:
df = df.fillna(0)
df.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,last_round,winner,head_diff,body_diff,leg_dif,pass_diff
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.0000,125.000000,125.000000,125.000000,125.000000
mean,0.184000,0.200000,0.552000,0.45600,2.392000,0.888000,0.216000,0.120000,4.440000,0.7760,0.099377,0.063367,0.057624,1.504000
std,0.498126,0.595683,1.117717,0.88449,4.120946,1.676286,0.603538,0.372437,0.928057,0.4186,0.167639,0.239342,0.235564,4.569577
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0000,-0.500000,-0.533333,-0.500000,-9.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.000000,1.0000,0.011909,-0.044872,-0.042553,0.000000
50%,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,5.000000,1.0000,0.094928,0.054348,0.000000,0.000000
75%,0.000000,0.000000,1.000000,1.00000,3.000000,1.000000,0.000000,0.000000,5.000000,1.0000,0.222617,0.212121,0.141379,2.000000
max,3.000000,4.000000,6.000000,4.00000,26.000000,9.000000,4.000000,2.000000,5.000000,1.0000,0.523148,0.944444,0.928571,26.000000


In [14]:
df['fight'] = df['R_fighter'] + " vs. " + df['B_fighter']
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,date,location,Fight_type,Winner,winner,head_diff,body_diff,leg_dif,pass_diff,fight
86,Max Holloway,Dustin Poirier,0,0,181 of 446,178 of 381,40%,46%,208 of 473,180 of 383,...,"April 13, 2019","Atlanta, Georgia, USA",UFC Interim Lightweight Title Bout,Dustin Poirier,0,-0.080527,0.030000,-0.375000,0,Max Holloway vs. Dustin Poirier
87,Kelvin Gastelum,Israel Adesanya,0,4,91 of 220,109 of 237,41%,45%,92 of 222,119 of 247,...,"April 13, 2019","Atlanta, Georgia, USA",UFC Interim Middleweight Title Bout,Israel Adesanya,0,-0.020323,0.054348,0.024725,0,Kelvin Gastelum vs. Israel Adesanya
149,Jon Jones,Anthony Smith,0,0,125 of 167,36 of 66,74%,54%,238 of 287,45 of 75,...,"March 02, 2019","Las Vegas, Nevada, USA",UFC Light Heavyweight Title Bout,Jon Jones,1,0.187500,0.232143,-0.042553,0,Jon Jones vs. Anthony Smith
150,Tyron Woodley,Kamaru Usman,0,0,34 of 51,141 of 194,66%,72%,60 of 79,336 of 390,...,"March 02, 2019","Las Vegas, Nevada, USA",UFC Welterweight Title Bout,Kamaru Usman,0,-0.109890,-0.006655,0.333333,-7,Tyron Woodley vs. Kamaru Usman
249,Valentina Shevchenko,Joanna Jedrzejczyk,0,0,89 of 191,78 of 250,46%,31%,118 of 223,124 of 300,...,"December 08, 2018","Toronto, Ontario, Canada",UFC Women's Flyweight Title Bout,Valentina Shevchenko,1,0.280145,0.075897,0.177833,3,Valentina Shevchenko vs. Joanna Jedrzejczyk


In [23]:
df2 = df[['pass_diff','head_diff','body_diff','leg_dif']]
df2.head()

,pass_diff,head_diff,body_diff,leg_dif
86,0,-0.080527,0.030000,-0.375000
87,0,-0.020323,0.054348,0.024725
149,0,0.187500,0.232143,-0.042553
150,-7,-0.109890,-0.006655,0.333333
249,3,0.280145,0.075897,0.177833


In [29]:
#read in scaler

file = open("data/fitted_scaler.pkl",'rb')
scaler = pickle.load(file)
file.close()

#this file is the data for title fights
champ_scaled = scaler.transform(df2)
champ_scaled_df = pd.DataFrame(champ_scaled, columns=df2.columns)
champ_scaled_df.head()

,pass_diff,head_diff,body_diff,leg_dif
0,-0.160607,-0.619416,0.028539,-1.466865
1,-0.160607,-0.280472,0.119257,0.041760
2,-0.160607,0.889550,0.781706,-0.212159
3,-2.174595,-0.784725,-0.108033,1.206495
4,0.702530,1.411130,0.199549,0.619612


In [42]:
#title fight labels:

champ_labels = df[['fight','R_fighter','B_fighter','Winner','winner']].reset_index(drop=True)
champ_labels.head()

,fight,R_fighter,B_fighter,Winner,winner
0,Max Holloway vs. Dustin Poirier,Max Holloway,Dustin Poirier,Dustin Poirier,0
1,Kelvin Gastelum vs. Israel Adesanya,Kelvin Gastelum,Israel Adesanya,Israel Adesanya,0
2,Jon Jones vs. Anthony Smith,Jon Jones,Anthony Smith,Jon Jones,1
3,Tyron Woodley vs. Kamaru Usman,Tyron Woodley,Kamaru Usman,Kamaru Usman,0
4,Valentina Shevchenko vs. Joanna Jedrzejczyk,Valentina Shevchenko,Joanna Jedrzejczyk,Valentina Shevchenko,1


In [43]:
result = pd.concat([champ_labels, champ_scaled_df], axis=1, sort=False)
result.head()

,fight,R_fighter,B_fighter,Winner,winner,pass_diff,head_diff,body_diff,leg_dif
0,Max Holloway vs. Dustin Poirier,Max Holloway,Dustin Poirier,Dustin Poirier,0,-0.160607,-0.619416,0.028539,-1.466865
1,Kelvin Gastelum vs. Israel Adesanya,Kelvin Gastelum,Israel Adesanya,Israel Adesanya,0,-0.160607,-0.280472,0.119257,0.041760
2,Jon Jones vs. Anthony Smith,Jon Jones,Anthony Smith,Jon Jones,1,-0.160607,0.889550,0.781706,-0.212159
3,Tyron Woodley vs. Kamaru Usman,Tyron Woodley,Kamaru Usman,Kamaru Usman,0,-2.174595,-0.784725,-0.108033,1.206495
4,Valentina Shevchenko vs. Joanna Jedrzejczyk,Valentina Shevchenko,Joanna Jedrzejczyk,Valentina Shevchenko,1,0.702530,1.411130,0.199549,0.619612


In [45]:
# save to pickle, for later use by plotly dash app.
result.to_pickle('data/title_fights.pkl')